<a href="https://colab.research.google.com/github/dtuleva/Recipe_Recommendation_System/blob/main/tfrs_featurization_recreate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Recreate TFRS tutorial: Using side features: feature preprocessing

One of the great advantages of using a deep learning framework to build recommender models is the freedom to build rich, flexible feature representations.

The first step in doing so is preparing the features, as raw features will usually not be immediately usable in a model.

For example:

- User and item ids may be strings (titles, usernames) or large, noncontiguous integers (database IDs).
- Item descriptions could be raw text.
- Interaction timestamps could be raw Unix timestamps.

These need to be appropriately transformed in order to be useful in building models:

- User and item ids have to be translated into embedding vectors: high-dimensional numerical representations that are adjusted during training to help the model predict its objective better.
- Raw text needs to be tokenized (split into smaller parts such as individual words) and translated into embeddings.
- Numerical features need to be normalized so that their values lie in a small interval around 0.

Fortunately, by using TensorFlow we can make such preprocessing part of our model rather than a separate preprocessing step. This is not only convenient, but also ensures that our pre-processing is exactly the same during training and during serving. This makes it safe and easy to deploy models that include even very sophisticated pre-processing.

In this tutorial, we are going to focus on recommenders and the preprocessing we need to do on the [MovieLens dataset](https://grouplens.org/datasets/movielens/). If you're interested in a larger tutorial without a recommender system focus, have a look at the full [Keras preprocessing guide](https://www.tensorflow.org/guide/keras/preprocessing_layers).

## The MovieLens dataset

Let's first have a look at what features we can use from the MovieLens dataset:

In [3]:
ratings_data = pd.read_csv("/content/mock-data_interaction.csv", index_col = 0)
recipes_data = pd.read_csv("/content/mock-data_recipe.csv", index_col = 0).drop(columns = ["nutritions", "image_url"])

In [6]:
ratings_data.dateLastModified = pd.to_datetime(ratings_data.dateLastModified).astype(int)

In [7]:
ratings_data.head() # TODO: parse inside the dataset or new csv

,user_id,recipe_id,rating,dateLastModified
0,8542392,222388,5,1492865203663000000
1,11174581,222388,5,1371743425960000000
2,8262477,222388,5,1423898871307000000
3,3574785,240488,5,1507400408973000000
4,12145410,240488,2,1515197169563000000


In [8]:
ratings_data.dtypes

user_id             int64
recipe_id           int64
rating              int64
dateLastModified    int64
dtype: object

In [9]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_data))
recipes = tf.data.Dataset.from_tensor_slices(dict(recipes_data))

In [10]:
import pprint


# for x in ratings.take(1).as_numpy_iterator():
#   pprint.pprint(x)

for x in recipes.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'aver_rate': 5.0,
 'cooking_directions': b"{'directions': u'Prep\\n5 m\\nCook\\n2 h 45 m\\nRead"
                       b'y In\\n11 h 50 m\\nPreheat oven to 200 degrees F (95 d'
                       b'egrees C).\\nSeason pork belly with paprika, salt, an'
                       b'd pepper. Tightly wrap pork twice in heavy-duty alum'
                       b'inum foil. Place on a baking sheet and bake in the p'
                       b'reheated oven for 2 1/2 hours. Turn off the oven; le'
                       b't pork rest in the oven for 1 hour. Remove meat from'
                       b' oven, leaving it wrapped in aluminum foil, and refr'
                       b'igerate at least 8 hours or overnight.\\nRemove pork '
                       b'from foil and slice across the grain in 1/4-inch thi'
                       b'ck slices. Working in batches, cook pork in a non-st'
                       b'ick skillet over medium heat until golden and crispe'
                       b"d, 6

In [11]:
for x in recipes.take(3).as_numpy_iterator():
  pprint.pprint(x["recipe_id"])

222388
240488
218939


In [12]:
test_recipe_ids = [222388, 240488, 218939]
test_user_ids = [8542392, 174581, 8262477]

In [13]:
for x in ratings.take(1).as_numpy_iterator():
  print(x)

{'user_id': 8542392, 'recipe_id': 222388, 'rating': 5, 'dateLastModified': 1492865203663000000}


There are a couple of key features here:

- Movie title is useful as a movie identifier.
- User id is useful as a user identifier.
- Timestamps will allow us to model the effect of time.

The first two are categorical features; timestamps are a continuous feature.

## Turning categorical features into embeddings

A [categorical feature](https://en.wikipedia.org/wiki/Categorical_variable) is a feature that does not express a continuous quantity, but rather takes on one of a set of fixed values.

Most deep learning models express these feature by turning them into high-dimensional vectors. During model training, the value of that vector is adjusted to help the model predict its objective better.

For example, suppose that our goal is to predict which user is going to watch which movie. To do that, we represent each user and each movie by an embedding vector. Initially, these embeddings will take on random values - but during training, we will adjust them so that embeddings of users and the movies they watch end up closer together.

Taking raw categorical features and turning them into embeddings is normally a two-step process:

1. Firstly, we need to translate the raw values into a range of contiguous integers, normally by building a mapping (called a "vocabulary") that maps raw values ("Star Wars") to integers (say, 15).
2. Secondly, we need to take these integers and turn them into embeddings.

### Defining the vocabulary

The first step is to define a vocabulary. We can do this easily using Keras preprocessing layers.

In [14]:
# movie_title_lookup = tf.keras.layers.StringLookup()

The layer itself does not have a vocabulary yet, but we can build it using our data.

In [15]:
# movie_title_lookup.adapt(ratings.map(lambda x: x["movie_title"]))

# print(f"Vocabulary: {movie_title_lookup.get_vocabulary()[:3]}")

Once we have this we can use the layer to translate raw tokens to embedding ids:

In [16]:
# movie_title_lookup(["Star Wars (1977)", "One Flew Over the Cuckoo's Nest (1975)"])

Note that the layer's vocabulary includes one (or more!) unknown (or "out of vocabulary", OOV) tokens. This is really handy: it means that the layer can handle categorical values that are not in the vocabulary. In practical terms, this means that the model can continue to learn about and make recommendations even using features that have not been seen during vocabulary construction.

### Using feature hashing # Nope

In fact, the `StringLookup` layer allows us to configure multiple OOV indices. If we do that, any raw value that is not in the vocabulary will be deterministically hashed to one of the OOV indices. The more such indices we have, the less likley it is that two different raw feature values will hash to the same OOV index. Consequently, if we have enough such indices the model should be able to train about as well as a model with an explicit vocabulary without the disdvantage of having to maintain the token list.

We can take this to its logical extreme and rely entirely on feature hashing, with no vocabulary at all. This is implemented in the `tf.keras.layers.Hashing` layer.

In [17]:
# We set up a large number of bins to reduce the chance of hash collisions.
num_hashing_bins = 10_000

recipe_id_hashing = tf.keras.layers.Hashing(
    num_bins=num_hashing_bins
)

We can do the lookup as before without the need to build vocabularies:

In [18]:
recipe_id_hashing(test_recipe_ids)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([4076, 7649, 1185])>

### Defining the embeddings

Now that we have integer ids, we can use the [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer to turn those into embeddings.

An embedding layer has two dimensions: the first dimension tells us how many distinct categories we can embed; the second tells us how large the vector representing each of them can be.

When creating the embedding layer for movie titles, we are going to set the first value to the size of our title vocabulary (or the number of hashing bins). The second is up to us: the larger it is, the higher the capacity of the model, but the slower it is to fit and serve.

In [19]:
# movie_title_embedding = tf.keras.layers.Embedding(
#     # Let's use the explicit vocabulary lookup.
#     input_dim=movie_title_lookup.vocab_size(),
#     output_dim=32
# )

In [20]:
recipe_id_embedding = tf.keras.layers.Embedding(
    # Let's use the explicit vocabulary lookup.
    input_dim = recipe_id_hashing.num_bins,
    output_dim=32
)

We can put the two together into a single layer which takes raw text in and yields embeddings.

In [21]:
recipe_id_model = tf.keras.Sequential([recipe_id_hashing, recipe_id_embedding])

Just like that, we can directly get the embeddings for our movie titles:

In [22]:
recipe_id_model(test_recipe_ids).shape

TensorShape([3, 32])

We can do the same with user embeddings:

In [23]:
user_id_lookup = tf.keras.layers.IntegerLookup()
user_id_lookup.adapt(ratings.map(lambda x: x["user_id"]))

user_id_embedding = tf.keras.layers.Embedding(user_id_lookup.vocabulary_size(), 32)

user_id_model = tf.keras.Sequential([user_id_lookup, user_id_embedding])

## Normalizing continuous features

Continuous features also need normalization. For example, the `timestamp` feature is far too large to be used directly in a deep model:

In [24]:
for x in ratings.take(3).as_numpy_iterator(): # datetime in ns in original dataset
  print(f"Timestamp: {x['dateLastModified']}.")

Timestamp: 1492865203663000000.
Timestamp: 1371743425960000000.
Timestamp: 1423898871307000000.


We need to process it before we can use it. While there are many ways in which we can do this, discretization and standardization are two common ones.

### Standardization

[Standardization](https://en.wikipedia.org/wiki/Feature_scaling#Standardization_(Z-score_Normalization)) rescales features to normalize their range by subtracting the feature's mean and dividing by its standard deviation. It is a common preprocessing transformation.

This can be easily accomplished using the [`tf.keras.layers.Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Normalization) layer:

In [25]:
for x in ratings.take(3).as_numpy_iterator():
  print((x["dateLastModified"]))

1492865203663000000
1371743425960000000
1423898871307000000


In [26]:
timestamp_normalization = tf.keras.layers.Normalization(
    axis=None
)
timestamp_normalization.adapt(ratings.map(lambda x: x["dateLastModified"]).batch(1024))

for x in ratings.take(3).as_numpy_iterator():
  print(f"Normalized timestamp: {timestamp_normalization(x['dateLastModified'])}.")

Normalized timestamp: [1.3276442].
Normalized timestamp: [0.42430067].
Normalized timestamp: [0.8132836].


In [27]:
timestamp_normalization([1492865203663000000, 1371743425960000000, 1423898871307000000])


<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.3276442 , 0.42430067, 0.8132836 ], dtype=float32)>

In [28]:
timestamp_normalization.mean

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.3148525e+18], dtype=float32)>

In [29]:
timestamp_normalization.variance

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.7977885e+34], dtype=float32)>

In [30]:
timestamp_normalization.invert = True

print(timestamp_normalization([1.3276442 , 0.42430067, 0.8132836 ]))

timestamp_normalization.invert = False


tf.Tensor([1.4928652e+18 1.3717434e+18 1.4238989e+18], shape=(3,), dtype=float32)


### Discretization

Another common transformation is to turn a continuous feature into a number of categorical features. This makes good sense if we have reasons to suspect that a feature's effect is non-continuous.

To do this, we first need to establish the boundaries of the buckets we will use for discretization. The easiest way is to identify the minimum and maximum value of the feature, and divide the resulting interval equally:

In [31]:
max_timestamp = ratings.map(lambda x: x["dateLastModified"]).reduce(
    tf.cast(0, tf.int64), tf.maximum).numpy().max()
min_timestamp = ratings.map(lambda x: x["dateLastModified"]).reduce(
    np.int64(1e9), tf.minimum).numpy().min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000)

print(f"Buckets: {timestamp_buckets[:3]}")

Buckets: [1.00000000e+09 1.52170384e+15 3.04340669e+15]


Given the bucket boundaries we can transform timestamps into embeddings:

In [32]:
timestamp_embedding_model = tf.keras.Sequential([
  tf.keras.layers.Discretization(timestamp_buckets.tolist()),
  tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32)
])

for timestamp in ratings.take(1).map(lambda x: x["dateLastModified"]).batch(1).as_numpy_iterator():
  print(f"Timestamp embedding: {timestamp_embedding_model(timestamp)}.")

Timestamp embedding: [[ 0.01451284  0.04052303  0.03796265  0.0368779  -0.04211018  0.00981431
  -0.01658794  0.03978509 -0.02062631 -0.02849709  0.03521403 -0.02704695
   0.03403516  0.02407378  0.01432372 -0.0304164  -0.00678953  0.00885507
  -0.04178615 -0.02547399 -0.03952188 -0.0204239   0.04537058  0.02627471
   0.03975539 -0.01073929 -0.00779893  0.04215908 -0.02701466  0.02438667
  -0.04121692 -0.01631363]].


## Processing text features

We may also want to add text features to our model. Usually, things like product descriptions are free form text, and we can hope that our model can learn to use the information they contain to make better recommendations, especially in a cold-start or long tail scenario.

While the MovieLens dataset does not give us rich textual features, we can still use movie titles. This may help us capture the fact that movies with very similar titles are likely to belong to the same series.

The first transformation we need to apply to text is tokenization (splitting into constituent words or word-pieces), followed by vocabulary learning, followed by an embedding.

The Keras [`tf.keras.layers.TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) layer can do the first two steps for us:

In [33]:
title_text = tf.keras.layers.TextVectorization()
title_text.adapt(recipes.map(lambda x: x["recipe_name"]))

Let's try it out:

In [34]:
for row in recipes.batch(1).map(lambda x: x["recipe_name"]).take(1):
  print(title_text(row))

tf.Tensor([[174  25]], shape=(1, 2), dtype=int64)


Each title is translated into a sequence of tokens, one for each piece we've tokenized.

We can check the learned vocabulary to verify that the layer is using the correct tokenization:

In [35]:
title_text.get_vocabulary()[40:45]

['in', 'ii', 'goulash', 'garlic', 'frittata']

This looks correct: the layer is tokenizing titles into individual words.

To finish the processing, we now need to embed the text. Because each title contains multiple words, we will get multiple embeddings for each title. For use in a donwstream model these are usually compressed into a single embedding. Models like RNNs or Transformers are useful here, but averaging all the words' embeddings together is a good starting point.

## Putting it all together

With these components in place, we can build a model that does all the preprocessing together.

### User model

The full user model may look like the following:

In [36]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        user_id_lookup,
        tf.keras.layers.Embedding(user_id_lookup.vocabulary_size(), 32),
    ])
    self.timestamp_embedding = tf.keras.Sequential([
      tf.keras.layers.Discretization(timestamp_buckets.tolist()),
      tf.keras.layers.Embedding(len(timestamp_buckets) + 2, 32)
    ])
    self.normalized_timestamp = tf.keras.layers.Normalization(
        axis=None
    )

  def call(self, inputs):

    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["dateLastModified"]),
        tf.reshape(self.normalized_timestamp(inputs["dateLastModified"]), (-1, 1))
    ], axis=1)

Let's try it out:

In [37]:
user_model = UserModel()

user_model.normalized_timestamp.adapt(
    ratings.map(lambda x: x["dateLastModified"]).batch(128))

for row in ratings.batch(1).take(1):
  print(f"Computed representations: {user_model(row)[0, :3]}")

Computed representations: [ 0.02198341 -0.03796301  0.00759153]


### Movie model
We can do the same for the movie model:

In [38]:
class RecipeModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      recipe_id_hashing,
      tf.keras.layers.Embedding(recipe_id_hashing.num_bins, 32)
    ])
    self.title_text_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(max_tokens=max_tokens),
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per title.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

  def call(self, inputs):
    return tf.concat([
        self.title_embedding(inputs["recipe_id"]),
        self.title_text_embedding(inputs["recipe_name"]),
    ], axis=1)

Let's try it out:

In [44]:
recipe_model = RecipeModel()

recipe_model.title_text_embedding.layers[0].adapt(
    recipes.map(lambda x: x["recipe_name"]))

for row in recipes.batch(1).take(1):
  print(f"Computed representation shape: {recipe_model(row).shape}")

Computed representation shape: (1, 64)


## Next steps

With the two models above we've taken the first steps to representing rich features in a recommender model: to take this further and explore how these can be used to build an effective deep recomender model, take a look at our Deep Recommenders tutorial.